<a href="https://colab.research.google.com/github/SY-256/llms-from-scratch/blob/main/notebooks/ch04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPTモデルを1から実装

In [ ]:
# GPTモデルの設定
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # 語彙のサイズ
    "context_length": 1024, # コンテキストの長さ
    "emb_dim": 768,         # 埋め込み次元数
    "n_heads": 12,          # Attentionヘッドの数
    "n_layers": 12,         # 層の数
    "drop_rate": 0.1,       # ドロップアウト率
    "qkv_bias": False       # クエリ、キー、バリューの計算にバイアスを使用するか
}

In [ ]:
# GPTプレイスホルダモデルアーキテクチャ
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):

    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        # TransformerBlockにはプレースホルダを使う
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        # LayerNormにもプレースホルダを使う(正規化層)
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

class DummyTransformerBlock(nn.Module):
    # 後ほど本物のTransformerBlockに置き換える
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        # 何もせずに入力を返すだけ
        return x

class DummyLayerNorm(nn.Module):
    # 後ほど本物のLAyerNormに置き換える
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()

    def forward(self, x):
        return x

In [ ]:
# GPTモデル用のトークナイザーを使用して
# 2つのテキスト入力からなるバッチをトークン化する
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

In [ ]:
# DummyGPTModelインスタンスを生成して
# トークン化したバッチbatchを入力として渡す
# モデルの出力は一般的にロジットと言う
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)

logits = model(batch)
print("Output shape: ", logits.shape)
print(logits)

## 4.2 層正規化を使って活性化を正規化する
- 勾配消失、勾配爆発を抑える
- layer nomarizationで層正規化を行い、ニューラルネットワーク層の出力（活性化）を平均0、分散1（単位分散）になるように調節
- 層正規化を行うことで、効果的な重みへの収束が早まり、訓練の一貫性と信頼性が確保される。
- 現代のTransformerアーキテクチャでは、layer normalaizationはMulti-head Attentionモジュールの前または後に適用するのが一般的

In [ ]:
# layer normalaizationの例
torch.manual_seed(123)

# それぞれ5つの次元（特徴量）を持つ2つの訓練サンプルを作成
batch_example = torch.randn(2, 5)
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out = layer(batch_example)
print(out)

出力結果の1行目は1つ目の入力に対する層の出力を表しており、2行目は2つ目の入力に対する層の出力を表している

In [ ]:
# Layer Normalizationの出力の平均と分散を調べる
mean = out.mean(dim=-1, keepdim=True)
var = out.var(dim=-1, keepdim=True)
print(f"Mean: {mean}")
print(f"Variance: {var}")

平均や分散の計算時にkeepdim=Trueを使うと、dimで指定された次元に沿ってテンソルを縮小したとしても、出力テンソルの次元数は入力テンソルと同じになる

In [ ]:
# 取得した層の出力にLayer Normalizationを適用
# 出力から平均を引き、分散の平方根（標準偏差）で割る -> 標準化する
out_norm = (out - mean) / torch.sqrt(var)
print(f"Normalization layer outputs:\n {out_norm}")

mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)
print(f"Mean:\n {mean}")
print(f"Variance:\n {var}")

In [ ]:
# 読みやすさ向上のため、指数表記をオフにすることもできる
torch.set_printoptions(sci_mode=False)
print(f"Mean: \n{mean}")
print(f"Variance: \n{var}")

In [ ]:
# 元に戻す
torch.set_printoptions(sci_mode=True)

In [ ]:
# Layer Normalizationの実装と適用をモジュール化

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False) # unbiased=True 有偏分散
        norm_x = (x - mean) / torch.sqrt(var + self.eps) # 標準化（ゼロ除算にならないようにepsを加える）
        return self.scale * norm_x + self.shift

### 有偏分散
`unbiased=False`を設定することで、実装依存の詳細を利用する

分散計算において、入力の数`n`で除算を行うことになる

ベッセル補正では、標本分散推定でのバイアスを調節するために分母を`n`ではなく`n-1`を使うのが一般的

LLMでは、埋め込みの次元数`n`が非常に大きいため、`n`と`n-1`の差は実質的に無視できる

また、オリジナルのTensorFlowのデフォルトの振る舞いを反映しているからという理由もあり有偏分散を用いる

In [ ]:
# LayerNormモジュールを実際にバッチに適用
ln = LayerNorm(emb_dim=5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim=-1, keepdim=True)
var = out_ln.var(dim=-1, unbiased=False, keepdim=True)
print(f"Mean: \n{mean}")
print(f"Variance: \n{var}")

バッチ正規化 -> バッチ次元に沿って正規化を行う

層正規化（`Layer Normalization`）-> 特徴量の次元に沿って正規化を行う

LLMは膨大な計算リソースが必要になることが多く、層正規化はバッチサイズとは関係なく各入力を正規化するため、そうしたシナリオで柔軟性と安定性が向上する

このため、分散学習を行う場合や、リソースに制約がある環境でモデルをデプロイする場合に、特に層正規化は効果が期待できる。

## 4.2 GELU活性化を使ってフィードフォワードネットワークを実装する
- GELUとSwiGLUの活性化関数が強い
- それぞれガウス線形ユニットとシグモイドゲート線形ユニットを組み込んでいる
- 単純なReLUの活性関数に比べてディープラーニングモデルの性能向上に寄与する

In [ ]:
# GELU活性化関数の実装
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [ ]:
# ReLU関数とGELU関数を描画して比較
import matplotlib.pyplot as plt

gelu, relu = GELU(), nn.ReLU()

x = torch.linspace(-3, 3, 100) # -3~3の範囲でデータを100個生成
y_gelu, y_relu = gelu(x), relu(x)
plt.figure(figsize=(8, 3))
for i, (y, label) in enumerate(zip([y_gelu, y_relu], ["GELU", "ReLU"]), 1):
    plt.subplot(1, 2, i)
    plt.plot(x, y)
    plt.title(f"{label} activate function")
    plt.xlabel("x")
    plt.ylabel(f"{label}(x)")
    plt.grid()

plt.tight_layout()
plt.show()

### GELU:
- 滑らかであり、モデルパラメータの微調整が可能であり、訓練時の最適化がよりスムーズに進む可能性がある
- 負の値に対して非ゼロの非常に小さい出力を許容するため、訓練プロセスで負の入力を受け取るニューロンが、正の入力ほどではないものの、依然として訓練プロセスに貢献できるようになる

### ReLU:
- ゼロで鋭く切り替わるため、非常に深いネットワークや複雑なアーキテクチャを持つネットワークでは最適化が難しくなる可能性がある
- 負の値では一律ゼロになるため、負の値を受け取ったニューロンが訓練プロセスに貢献できなくなる

In [ ]:
# フォードフォワードニューラルネットワークモジュール
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), # 1つ目の線形層で入力の次元から4倍にして出力
            GELU(), # GELU活性化関数
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), # 2つ目の線形層で4倍にした入力の次元数を4分の1の元の次元数に戻して出力
        )

    def forward(self, x):
        return self.layers(x)

FeedForwardモジュールは2つのLinear層とGELU活性化関数からなる小さなニューラルネットワーク

In [ ]:
# FeedForwardネットワークを使ってみる
ffn = FeedForward(GPT_CONFIG_124M)

x = torch.rand(2, 3, 768) # バッチ次元2のサンプル入力を作成
out = ffn(x)
print(out.shape)

出力テンソルの形状が入力テンソルと同じ

FeedForwardモジュールは、データから学習して汎用化するモデルの能力を向上させる上で重要な役割を果たす

1つ目の線形層で埋め込みの次元数が高次元空間に4倍に拡張され、GELU活性化関数によって非線形変換が適用され、2つ目の線形層で元の次元に縮小される-> より表現力のある空間の探索が可能になる

入力と出力の次元が均一であるため、アーキテクチャも単純になる -> 複数の層を積み重ねることが可能になり、層の間で次元を調節する必要がなく、モデルのスケーラビリティが向上する


## 4.4 ショートカット接続（残差接続）を追加する
- 勾配消失問題を軽減する
- 勾配: 訓練中に重みを更新する指標のようなもの
- 勾配消失問題: 勾配がネットワークの後ろの層に伝播する過程で徐々に小さくなり、入力に近い最小にある層を誤差逆伝播（バックプロパゲーション）したときに上手く訓練できなくなる問題

In [ ]:
# forward()にショートカット接続を追加したニューラルネットワーク
class ExampleDeepNeuralNetwork(nn.Module):
    def __init__(self, layer_size, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        # 5つの層を実装
        self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_size[0], layer_size[1]), GELU()),
            nn.Sequential(nn.Linear(layer_size[1], layer_size[2]), GELU()),
            nn.Sequential(nn.Linear(layer_size[2], layer_size[3]), GELU()),
            nn.Sequential(nn.Linear(layer_size[3], layer_size[4]), GELU()),
            nn.Sequential(nn.Linear(layer_size[4], layer_size[5]), GELU()),
        ])

    def forward(self, x):
        for layer in self.layers:
            layer_output = layer(x) # 現在の層の出力を計算
            if self.use_shortcut and x.shape == layer_output.shape: # ショートカットを適用できるかチェック
                x = x + layer_output # ショートカット適用できる場合: 入力をそのまま層の出力結果に加える
            else:
                x = layer_output
        return x

# 層ごとの勾配の変化を表示
def print_grandients(model, x):
    # Forward pass
    output = model(x)
    target = torch.tensor([[0.]])

    # ターゲットに対する損失の計算
    loss = nn.MSELoss()
    loss = loss(output, target)

    # 勾配の誤差逆伝播
    # モデルの訓練時に必要な損失関数の勾配を自動で行える
    loss.backward()

    for name, param in model.named_parameters():
        if "weight" in name:
            # 勾配の絶対値平均
            print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")

In [ ]:
# ショートカット接続なしのニューラルネットワーク
layer_sizes = [3, 3, 3, 3, 3, 1] # 各層の大きさ
sample_input = torch.tensor([[1., 0., -1.]])

torch.manual_seed(123) # 再現性を確保するために重みの初期値に対して乱数シードを設定
model_without_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes, use_shortcut=False
)
print_grandients(model_without_shortcut, sample_input)

最後の層（`layers.4.0`）から最初の層（`layers.0.0`）に進むに従って勾配が小さくなっている -> 勾配消失問題

In [ ]:
# ショートカット接続を行うニューラルネットワーク
torch.manual_seed(123)
model_shortcut = ExampleDeepNeuralNetwork(
    layer_sizes, use_shortcut=True
)
print_grandients(model_shortcut, sample_input)

最初の層（`layers.0.0`）に進む過程で勾配の値が安定し、消失するほど小さな値に縮小していない -> 勾配消失問題が起こりづらい

- ショートカット接続によって層から層への勾配の流れが一貫したものになり、訓練をより効果的に進めることができる

## 4.5 TransformerブロックでAttention層と線形層を接続する
- Transformerブロック: Multi-head Attention, 層正規化, ドロップアプト, フィードフォワード層, GELU活性化関数を組み合わせる
- 1億2,400万パラメータのGPT-2アーキテクチャではTransformerブロックを12回繰り返している

In [ ]:
# 3章のMulti-head Attentionコンポーネント
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0) # 出力に次元数がヘッド数と割り切れない場合は警告出す

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # 出力次元数をヘッド数で分割

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out) # Linear層を使ってヘッドの出力を組み合わせる
        self.dropout = nn.Dropout(dropout)
        # maskを作成 (context_lengthの形状に従う)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)

        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape # テンソルの形状は(batch, num_tokens, d_out)

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # num_heads次元を追加して行列えお暗黙的に分割
        # 最後の次元を展開し、形状を(batch, num_tokens, d_out) -> (batch, num_tokens, num_heads, head_dim)に変換
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # 形状を(batch, num_tokens, num_heads, head_dim) ->
        # (batch, num_heads, num_tokens, head_dim)に変換
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3) # 各ヘッドのドット積を計算

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # マスクをトークン数で切り捨て

        attn_scores.masked_fill(mask_bool, -torch.inf) # Attentionスコアを埋めるためにマスクを使う

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2) # テンソルの形状は(batch, num_tokens, n_heads, head_dim)
        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        ) # self.d_out = self.num_heads * self.head_dimに基づいてヘッドを結合

        context_vec = self.out_proj(context_vec) # 線形射影を追加

        return context_vec

In [ ]:
# GPTアーキテクチャのTransformerブロックコンポーネント
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"]
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):

        shortcut = x # Attentionブロックのショートカット接続(入力のオリジナル)
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut # 元の入力をショートカット接続として入力

        shortcut = x # フィードフォワードブロックのショートカット接続(前段の処理結果)
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x

Multi-head Attentionブロックでは、Self-Attentionメカニズムが入力シーケンスの要素間の関係を識別する。フィードフォワードネットワークでは、それぞれの位置でデータが個別に変更される。
- Multi-head Attentionとフィードフォワードネットワークの組み合わせによって、入力の微妙な違いを理解して処理できるようになる
- 複雑なデータパターンを処理するモデルの全体的な能力が強化される

- 層正規化（`LayerNorm`）はMulti-head Attentionとフィードフォワードネットワークの2つのコンポーネントの前に適用される`Pre-LayerNorm`
- ドロップアウトは、モデルを正則化して過剰適合を防ぐために、これらのコンポーネントの後に適用される
- オリジナルのTransformerではMulti-head Attentionとフィードフォワードネットワークの後に層正規化（`LayerNorm`）を適用する`Post-LayerNorm`
- Post-LayerNormはしばしば学習のダイナミクス（そのものが持つ力強さ、能力）に悪影響を及ぼす